In [3]:
import numpy as np
from numpy.random import seed
seed(42)

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

In [2]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3
SHADOW_DATASET_SIZE = 1000
ATTACK_TEST_DATASET_SIZE = 4000


target_epochs = 12
attack_epochs = 12
num_shadows = 10

In [4]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")[:10000]
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")[:10000]
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)

def target_model_fn():
    """The architecture of the target (victim) model.

    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = tf.keras.models.Sequential()

    model.add(
        layers.Conv2D(
            32,
            (3, 3),
            activation="tanh",
            padding="same",
            input_shape=(WIDTH, HEIGHT, CHANNELS),
        )
    )
    #model.add(layers.Conv2D(32, (3, 3), activation="tanh"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    #model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, (3, 3), activation="tanh", padding="same"))
    #model.add(layers.Conv2D(64, (3, 3), activation="tanh"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    #model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())

    model.add(layers.Dense(128, activation="tanh"))
    #model.add(layers.Dropout(0.5))

    model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
    model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

def attack_model_fn():
    """Attack model that takes target model predictions and predicts membership.

    Following the original paper, this attack model is specific to the class of the input.
    AttachModelBundle creates multiple instances of this model for each class.
    """
    model = tf.keras.models.Sequential()

    model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [20]:
(X_train, y_train), (X_test, y_test) = get_data()

# Train the target model.
print("Training the target model...")
target_model = target_model_fn()
target_model.fit(X_train, y_train, epochs=target_epochs, validation_split=0.2, verbose=True)

Training the target model...
Train on 8000 samples, validate on 2000 samples
Epoch 1/12
8000/8000 [==============================] - 35s 4ms/sample - loss: 1.7160 - acc: 0.3824 - val_loss: 1.4960 - val_acc: 0.4685
Epoch 2/12
8000/8000 [==============================] - 33s 4ms/sample - loss: 1.3504 - acc: 0.5188 - val_loss: 1.3497 - val_acc: 0.5360
Epoch 3/12
8000/8000 [==============================] - 35s 4ms/sample - loss: 1.1183 - acc: 0.6104 - val_loss: 1.2361 - val_acc: 0.5720
Epoch 4/12
8000/8000 [==============================] - 37s 5ms/sample - loss: 0.9425 - acc: 0.6685 - val_loss: 1.2723 - val_acc: 0.5700
Epoch 5/12
8000/8000 [==============================] - 33s 4ms/sample - loss: 0.7512 - acc: 0.7445 - val_loss: 1.2482 - val_acc: 0.5855
Epoch 6/12
8000/8000 [==============================] - 32s 4ms/sample - loss: 0.5552 - acc: 0.8171 - val_loss: 1.3282 - val_acc: 0.5835
Epoch 7/12
8000/8000 [==============================] - 35s 4ms/sample - loss: 0.3523 - acc: 0.8969 -

In [29]:
target_model.save("target_cifar10.h5")

In [ ]:
from tensorflow.keras.models import load_model

target_model = load_model('non_federated_cifar10.h5')


In [35]:
# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=num_shadows
)

# We assume that attacker's data were not seen in target's training.
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    X_test, y_test, test_size=0.5)
print(attacker_X_train.shape, attacker_X_test.shape)

print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=target_epochs,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test)
    )
)

(5000, 32, 32, 3) (5000, 32, 32, 3)
Training the shadow models...
Train on 1000 samples, validate on 5000 samples
Epoch 1/12
1000/1000 [==============================] - 9s 9ms/sample - loss: 2.1918 - acc: 0.2150 - val_loss: 2.1064 - val_acc: 0.2338
Epoch 2/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.8171 - acc: 0.3550 - val_loss: 1.8500 - val_acc: 0.3548
Epoch 3/12
1000/1000 [==============================] - 7s 7ms/sample - loss: 1.6174 - acc: 0.4470 - val_loss: 1.7858 - val_acc: 0.3518
Epoch 4/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.4222 - acc: 0.5080 - val_loss: 1.7772 - val_acc: 0.3778
Epoch 5/12
1000/1000 [==============================] - 7s 7ms/sample - loss: 1.2087 - acc: 0.5700 - val_loss: 1.9375 - val_acc: 0.3356
Epoch 6/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.0565 - acc: 0.6340 - val_loss: 1.9189 - val_acc: 0.3586
Epoch 7/12
1000/1000 [==============================] - 8s 8ms/sample 

Epoch 11/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 0.0779 - acc: 1.0000 - val_loss: 2.3234 - val_acc: 0.4048
Epoch 12/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 0.0404 - acc: 1.0000 - val_loss: 2.3357 - val_acc: 0.4110
Train on 1000 samples, validate on 5000 samples
Epoch 1/12
1000/1000 [==============================] - 9s 9ms/sample - loss: 2.1385 - acc: 0.2300 - val_loss: 2.0133 - val_acc: 0.2848
Epoch 2/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.7957 - acc: 0.3670 - val_loss: 1.8114 - val_acc: 0.3528
Epoch 3/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.5976 - acc: 0.4540 - val_loss: 1.7624 - val_acc: 0.3524
Epoch 4/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.3284 - acc: 0.5470 - val_loss: 1.8139 - val_acc: 0.3828
Epoch 5/12
1000/1000 [==============================] - 8s 8ms/sample - loss: 1.1082 - acc: 0.6190 - val_loss: 1.7629 - val_acc: 0.391

In [36]:
from sklearn.metrics import classification_report

# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=attack_epochs, verbose=True)
)

Training the attack models...
Epoch 1/12
2039/2039 [==============================] - 1s 609us/sample - loss: 0.5538 - acc: 0.7430
Epoch 2/12
2039/2039 [==============================] - 0s 100us/sample - loss: 0.3670 - acc: 0.8509
Epoch 3/12
2039/2039 [==============================] - 0s 117us/sample - loss: 0.3472 - acc: 0.8602
Epoch 4/12
2039/2039 [==============================] - 0s 102us/sample - loss: 0.3456 - acc: 0.8646
Epoch 5/12
2039/2039 [==============================] - 0s 107us/sample - loss: 0.3421 - acc: 0.8622
Epoch 6/12
2039/2039 [==============================] - 0s 114us/sample - loss: 0.3383 - acc: 0.8637
Epoch 7/12
2039/2039 [==============================] - 0s 117us/sample - loss: 0.3374 - acc: 0.8700
Epoch 8/12
2039/2039 [==============================] - 0s 108us/sample - loss: 0.3333 - acc: 0.8676
Epoch 9/12
2039/2039 [==============================] - 0s 96us/sample - loss: 0.3368 - acc: 0.8700
Epoch 10/12
2039/2039 [==============================] - 0s 10

In [43]:
# Test the success of the attack.

# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = attacker_X_test[:ATTACK_TEST_DATASET_SIZE], attacker_y_test[:ATTACK_TEST_DATASET_SIZE]

# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)

print(attack_accuracy)
print (classification_report(real_membership_labels, attack_guesses))

0.79975
             precision    recall  f1-score   support

        0.0       1.00      0.60      0.75      4000
        1.0       0.71      1.00      0.83      4000

avg / total       0.86      0.80      0.79      8000

